<h1 style="color: navy;text-align: center;">Signal Processing Performance Profiling</h1>
<hr>
<p>
This notebook is used to examine the performance of the code which generates the signal processing features.
</p>

<h3 style="color: navy">Set-up for Profiling</h3>

In [1]:
# Imports

import numpy as np
import h5py
import os

from pathlib import Path

from src.pose_estimation import open_pose_file

# Bring in some base feature modules.
from src.feature_extraction.base_features import (moments as Moments, 
                                                  point_speeds as PointSpeeds)

In [2]:
# Install the line profiler module and load it.

%pip install line_profiler
%load_ext line_profiler


Note: you may need to restart the kernel to use updated packages.


In [3]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
%lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These statis

<p>
The <b style="color: navy;">pose_path</b> variable should point to the pose_est h5 file of interest.  In my case, the test file resides in the data directory.
</p>

In [4]:
# Open pose file and create pose estimate object. 

pose_path = "data" / Path("B6J_MDB0054_pose_est_v6.h5")

if os.path.isfile(pose_path):
    pose = open_pose_file(pose_path)

<p>
Test that the moments (or other base feature of interest) can be initialized and computed per frame.
</p>

In [5]:
test_identity = 1

moment = Moments.Moments(pose, pose.cm_per_pixel)

assert moment._name == 'moments'

moment_features = moment.per_frame(test_identity)

assert moment_features.shape[0] == pose.num_frames

<h3 style="color: navy;">Profiling with the line profiler</h3>
<p>
The <b style="color: navy;">nframes</b> variable slices the array down to a size such that the signal processing features can be computed faster.
</p>
<h4 style="color: navy;">i. <b>signal_processing</b> Profile</h4>

In [6]:
window_size = 5

nframes = 10

%lprun -T signal_processing_profile.txt -f moment.signal_processing moment.signal_processing(test_identity, window_size, moment_features[:nframes, ...], nframes)

c:\Users\sebasn\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1964: UserWarning: nperseg = 16 is greater than input length  = 11, using nperseg = 11
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\sebasn\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:755: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
c:\Users\sebasn\Documents\Programs\JABS-behavior-classifier\src\feature_extraction\feature_base_class.py:245: UserWarning: Warning: converting a masked element to nan.
  return np.array([



*** Profile printout saved to text file 'signal_processing_profile.txt'. 


Timer unit: 1e-07 s

Total time: 0.79388 s
File: c:\Users\sebasn\Documents\Programs\JABS-behavior-classifier\src\feature_extraction\feature_base_class.py
Function: signal_processing at line 179

Line #      Hits         Time  Per Hit   % Time  Line Contents
   179                                               def signal_processing(
   180                                                   self, identity: int, window_size: int,
   181                                                   feature_values: np.ndarray,
   182                                                   nframes: int
   183                                               ) -> typing.Dict:
   184                                                   """
   185                                                   This method combines the helper functions, compute_frequency_features
   186                                                   and get_frequency_feature for better line profiling.
   187                                        

<h4 style="color: navy;">ii. <b>get_frequency_feature</b> Profile</h4>

In [9]:
nframes = 500

mx = moment.build_mx_array(test_identity, window_size, moment_features[:nframes, ...], nframes)

curr_frame = 0


%lprun -T get_frequency_feature_profile.txt -f moment.get_frequency_feature moment.get_frequency_feature(mx[curr_frame,:])


*** Profile printout saved to text file 'get_frequency_feature_profile.txt'. 


Timer unit: 1e-07 s

Total time: 0.0050886 s
File: c:\Users\sebasn\Documents\Programs\JABS-behavior-classifier\src\feature_extraction\feature_base_class.py
Function: get_frequency_feature at line 230

Line #      Hits         Time  Per Hit   % Time  Line Contents
   230                                               def get_frequency_feature(
   231                                                       self, wave: np.ndarray, a: np.ndarray = _a, b: np.ndarray = _b,
   232                                                       samplerate: float = _samplerate) -> dict:
   233                                                   """
   234                                                   :param wave: an array representing the signals for each frame for a
   235                                                       given identity for a particular window size.
   236                                                   :param a: The denominator coefficient vector of the filter.
   237             